In [6]:
#import
import pandas as pd
import requests
import seaborn
from datetime import datetime

In [7]:
def date_string_to_unix_timestamp(date_string, date_format="%Y-%m-%d %H:%M:%S"):
    """
    Converts a datestring to a unix timestamp.
    
    Args:
        date_string (str): The date string to convert.
        date_format (str, optional): The format of the date string.
                                     Defaults to "%Y-%m-%d %H:%M:%S".

    Returns:
        float: The Unix timestamp (seconds since epoch) as a float.
               Returns None if the date string cannot be parsed.
    """

    try:
        dt_object = datetime.strptime(date_string, date_format)
        unix_timestamp = int(dt_object.timestamp())
        return unix_timestamp
    except ValueError as e:
        print(f"Error: Could not parse the date string. Check your format code. Details: {e}")
        raise #stops the notebook from running 

In [8]:
#define the api endpoints for last.fm
url = "http://ws.audioscrobbler.com/2.0/"

#define parameters
API_KEY = "81795854e49b1656fb706caeb7586f05"
USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64; rv:143.0) Gecko/20100101 Firefox/143.0"
USER = "qst4"
FROM = "2010-01-01 00:00:00"
TO = "2010-06-01 00:00:00"

#set parameters for the API request
params = {
    "method" : "user.getweeklyartistchart",
    "api_key" : API_KEY,
    "user" : USER,
    "format" : "json",
    "user-agent" : "Mozilla/5.0 (X11; Linux x86_64; rv:143.0) Gecko/20100101 Firefox/143.0",
    "from" : date_string_to_unix_timestamp(FROM),
    "to" : date_string_to_unix_timestamp(TO)
}

In [9]:
#make the request
response = requests.get(url, params=params)

#check if the request was successful
if response.status_code == 200:
    data = response.json()
    # Extract the top artist
    artist_data = data['weeklyartistchart']['artist'][0]
    print(f"Artist: {artist_data['name']}")
    print(f"Playcount: {artist_data['playcount']}")
    print(f"URL: {artist_data['url']}")
else:
    print(f"Error: {response.status_code} - {response.text}") #relook this

Artist: Danilo Caymmi
Playcount: 135
URL: https://www.last.fm/music/Danilo+Caymmi


In [10]:
# Create a list of dictionaries for the DataFrame
from_to_chart = [
    {
        'Rank': i + 1,
        'Artist': artist['name'],
        'Playcount': artist['playcount'],
        'Last.fm URL': artist['url']
    }
    for i, artist in enumerate(data['weeklyartistchart']['artist'])
]

# Create the DataFrame
df = pd.DataFrame(from_to_chart)

# Display the DataFrame
df

,Rank,Artist,Playcount,Last.fm URL
0,1,Danilo Caymmi,135,https://www.last.fm/music/Danilo+Caymmi
1,2,Madlib,115,https://www.last.fm/music/Madlib
2,3,J Dilla,109,https://www.last.fm/music/J+Dilla
3,4,Sade,103,https://www.last.fm/music/Sade
4,5,Grant Green,82,https://www.last.fm/music/Grant+Green
...,...,...,...,...
453,454,Toto,1,https://www.last.fm/music/Toto
454,455,U2,1,https://www.last.fm/music/U2
455,456,Vanessa Williams,1,https://www.last.fm/music/Vanessa+Williams
456,457,Weldon Irvine,1,https://www.last.fm/music/Weldon+Irvine
